<a href="https://colab.research.google.com/github/chris00234/NLP/blob/main/transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [9]:
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
dataset = load_dataset("cnn_dailymail", "3.0.0")
print(f"Features in cnn_dailymail: {dataset['train'].column_names}")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Features in cnn_dailymail: ['article', 'highlights', 'id']


In [12]:
sample = dataset["train"][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}): ')
print(sample["highlights"])


Article (excerpt of 500 characters, total length: 4051):

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary (length: 281): 
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


In [19]:
sample_text = dataset["train"][1]["article"][:1000]
summaries = {}
sample_text

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

In [14]:
def baseline_summary_three_sent(text):
  return "\n".join(sent_tokenize(text)[:3])

In [16]:
summaries['baseline'] = baseline_summary_three_sent(sample_text)
summaries['baseline']

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."'

### GPT2

In [17]:
from transformers import pipeline,set_seed

set_seed(42)

pipe = pipeline('text-generation', model='gpt2-medium')

gpt2_query = sample_text + "\nTL;DR:\n"

pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [18]:
pipe_out

[{'generated_text': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and

In [20]:
pipe_out[0]['generated_text'][len(gpt2_query) :]

'To get to the jail that holds the mentally ill, visit our Behind the Scenes blog -- Click here The story doesn\'t end there:\xa0 In 2014, a judge ordered the jail to provide treatment for 40 mental-health detainees in the mental-health unit, as part of a $22,000 settlement.\nInmates in the mental-health unit at Miami-Dade County\'s jail are often locked in a cell that\'s usually just like any other in the facility and can become chaotic. Mental health unit employees are often required to leave the jail and drive to the facility, instead of coming in to work with the inmates themselves. Most mental-health detainees spend hours a day sleeping in front of the wall by the pool, waiting for treatment to show up.\nThere are two more stories from the Inside the Tincup Jail series:\n\xa0\xa0\xa0 The "no contact" policy:\xa0 At an overcrowded state mental health facility in Miami-Dade County, the rules allow the police to try to convince the jail staff to allow a person they believe is mentall

In [21]:
summaries['gpt2'] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query) :]))

### T5

In [22]:
pipe = pipeline('summarization', model='t5-small')

pipe_out = pipe(sample_text)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [27]:
pipe_out

[{'summary_text': "inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court . most often, they face drug charges or charges of assaulting an officer . mentally ill people become more paranoid, delusional, and less likely to follow dir ."}]

In [28]:
summaries['t5'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))

### BART

In [29]:
pipe = pipeline('summarization', model='facebook/bart-large-cnn')

pipe_out = pipe(sample_text)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [30]:
pipe_out

[{'summary_text': 'Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most severe mental illnesses are incarcerated. Most often, they face drug charges or charges of assaulting an officer. Judge Steven Leifman says the arrests often result from confrontations with police.'}]

In [31]:
summaries['bart'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))

### PEGASUS

In [32]:
pipe = pipeline('summarization', model='google/pegasus-xsum')

pipe_out = pipe(sample_text)

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [33]:
pipe_out

[{'summary_text': 'An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.'}]

In [34]:
summaries['pegasus'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))

## comparing different summaries

In [35]:
print("GROUND TRUTH")
print(dataset['train'][1]['highlights'])

for model_name in summaries:
  print(model_name.upper())
  print(summaries[model_name])

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
GPT2
To get to the jail that holds the mentally ill, visit our Behind the Scenes blog -- Click here The story doesn't end there:  In 2014, a judge ordered the jail to provide treatment for 40 mental-health detainees in the mental-health unit, as part of a $22,000 settlement.
Inmates in the mental-health unit at Miami-Dade County's jail are often locked in a cell that's usually just like any other in the facility and can become chaotic.
Mental health unit employees are often required to leave the jail and drive to the facility, instead of coming in to work with the inmates themselves.
Most mental-health detainees spend hours a day sleeping in front of the wall by the pool, waiting for treatment to

### SacreBLEU

In [36]:
from datasets import load_metric

bleu_metric = load_metric('sacrebleu')

<ipython-input-36-07e747a21c2e>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('sacrebleu')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [42]:
bleu_metric.add(prediction = [summaries['gpt2']], reference = [dataset['train'][1]['highlights']])
results = bleu_metric.compute(smooth_method = 'floor', smooth_value = 0)
results['precision'] = [np.round(p, 2) for p in results['precisions']]

pd.DataFrame.from_dict(results, orient= 'index', columns = ['Value'])

,Value
score,0.0
counts,"[25, 2, 0, 0]"
totals,"[299, 298, 297, 296]"
precisions,"[8.361204013377927, 0.6711409395973155, 0.0, 0.0]"
bp,1.0
sys_len,299
ref_len,57
precision,"[8.36, 0.67, 0.0, 0.0]"


## ROUGE

In [43]:
rouge_metric = load_metric('rouge')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
